# 4.1 データセットと前処理 — ハードコーディング版

このノートブックでは、第4.1節の内容に沿って、モデル非依存の前処理パイプラインをハードコーディングで実装します。
- データ取得（Hugging Face datasets: globis-university/aozorabunko-clean の train）
- 文字列正規化（Unicode NFKC、空白・改行整形）
- 簡易クリーニング（制御・ゼロ幅文字の除去）
- 厳密重複除去（SHA1）
- 長さ・低情報フィルタ
- シャッフルと train/val 分割
- TXT/JSONL/マニフェストの保存（出力先: data/aozora）
なお、トークナイザの学習・保存は4.3節で実施します。

入出力パスや閾値は下のセルで固定値として定義しています。

In [12]:
from __future__ import annotations

import json
import hashlib
import random
import re
import unicodedata
from pathlib import Path
from typing import List

# ハードコーディングされた設定
OUTPUT_DIR = Path('data')  # 出力先（HFデータからの前処理成果物）
TRAIN_RATIO = 0.98
MIN_CHARS = 50
MAX_CHARS = 0  # 0なら無効
UNIQUE_CHAR_RATIO_MIN = 0.01  # 0なら無効
SEED = 42

RE_MULTISPACE = re.compile(r'\s+')
RE_MULTI_NL = re.compile(r'\n{2,}')

def normalize_text(text: str) -> str:
    """基本的な正規化：
    - 改行をLFへ統一
    - Unicode NFKC
    - 行内の空白連続を1つに、前後空白の削除
    - 連続空行の圧縮
    """
    if not text:
        return ''
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    text = unicodedata.normalize('NFKC', text)
    lines = []
    for line in text.split('\n'):
        line = line.strip()
        if line:
            line = RE_MULTISPACE.sub(' ', line)
        lines.append(line)
    text = '\n'.join(lines)
    text = RE_MULTI_NL.sub('\n\n', text)
    return text.strip()

def simple_clean_hooks(text: str) -> str:
    """簡易クリーニング：ゼロ幅文字や制御文字の除去（必要に応じてルールを追加）
    """
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F]', '', text)
    return text

def exact_deduplicate(docs: List[str]) -> List[str]:
    """SHA1による文書単位の厳密重複除去"""
    seen = set()
    uniq = []
    for d in docs:
        key = hashlib.sha1(d.encode('utf-8')).hexdigest()
        if key in seen:
            continue
        seen.add(key)
        uniq.append(d)
    return uniq

def length_filter(docs: List[str], min_chars: int | None, max_chars: int | None, unique_char_ratio_min: float | None):
    out = []
    for d in docs:
        n = len(d)
        if min_chars is not None and n < min_chars:
            continue
        if max_chars is not None and n > max_chars:
            continue
        if unique_char_ratio_min is not None and n > 0:
            ratio = len(set(d)) / n
            if ratio < unique_char_ratio_min:
                continue
        out.append(d)
    return out

def shuffle_and_split(docs: List[str], train_ratio: float, seed: int):
    idx = list(range(len(docs)))
    rng = random.Random(seed)
    rng.shuffle(idx)
    cut = int(len(idx) * train_ratio)
    train = [docs[i] for i in idx[:cut]]
    val = [docs[i] for i in idx[cut:]]
    return train, val

def save_text(path: Path, docs: List[str]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open('w', encoding='utf-8') as f:
        for d in docs:
            f.write(d.strip())
            f.write('\n\n')  # 空行区切り

def save_jsonl(path: Path, docs: List[str]):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open('w', encoding='utf-8') as f:
        for d in docs:
            f.write(json.dumps({'text': d}, ensure_ascii=False))
            f.write('\n')

def write_manifest(path: Path, manifest: dict):
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open('w', encoding='utf-8') as f:
        json.dump(manifest, f, ensure_ascii=False, indent=2)


## データ取得（Hugging Face datasets）
`globis-university/aozorabunko-clean` の train split を取得します。

In [13]:
from datasets import load_dataset
TEXT_COL = 'text'
ds = load_dataset('globis-university/aozorabunko-clean')
texts = ds['train'][TEXT_COL]
len(texts), (texts[0][:80] if texts else '(no text)')


(16951,
 '深いおどろきにうたれて、\n名高いウェストミンスターに\n真鍮や石の記念碑となって\nすべての王侯貴族が集まっているのをみれば、\n今はさげすみも、ほこりも、見栄もない')

## 正規化・クリーニング
NFKC正規化→簡易クレンジング→空白・改行整形を適用します。

In [14]:
raw_docs = []
for t in texts:
    text = normalize_text(simple_clean_hooks(t))
    if text:
        raw_docs.append(text)
len(raw_docs), sum(len(d) for d in raw_docs)


(16950, 229433847)

## 重複除去（厳密）
SHA1キーで厳密な重複を除去します。

In [15]:
dedup_docs = exact_deduplicate(raw_docs)
len(dedup_docs)


16950

## フィルタリング
- `MIN_CHARS` 未満を除外
- `MAX_CHARS` が正なら上限適用
- `UNIQUE_CHAR_RATIO_MIN` が正ならユニーク文字比の閾値を適用

In [16]:
max_chars = MAX_CHARS if MAX_CHARS and MAX_CHARS > 0 else None
ucrm = UNIQUE_CHAR_RATIO_MIN if UNIQUE_CHAR_RATIO_MIN and UNIQUE_CHAR_RATIO_MIN > 0 else None
filtered_docs = length_filter(dedup_docs, MIN_CHARS, max_chars, ucrm)
len(filtered_docs)


16799

## 分割（train/val）
シャッフルの上、`TRAIN_RATIO` で分割します。

In [17]:
train_docs, val_docs = shuffle_and_split(filtered_docs, TRAIN_RATIO, SEED)
len(train_docs), len(val_docs)


(16463, 336)

## 保存（TXT/JSONL/manifest）
各文書はTXTでは空行区切り、JSONLでは1行1文書（{text: ...}）で保存します。
保存先は data/ です。

In [18]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
save_text(OUTPUT_DIR / 'train.txt', train_docs)
save_text(OUTPUT_DIR / 'val.txt', val_docs)
save_jsonl(OUTPUT_DIR / 'train.jsonl', train_docs)
save_jsonl(OUTPUT_DIR / 'val.jsonl', val_docs)
manifest = {
    'total_docs': len(raw_docs),
    'total_chars': int(sum(len(d) for d in raw_docs)),
    'after_dedup_docs': len(dedup_docs),
    'after_filter_docs': len(filtered_docs),
    'train_docs': len(train_docs),
    'val_docs': len(val_docs),
    'params': {
        'train_ratio': TRAIN_RATIO,
        'min_chars': MIN_CHARS,
        'max_chars': MAX_CHARS,
        'unique_char_ratio_min': UNIQUE_CHAR_RATIO_MIN,
        'seed': SEED,
    },
}
write_manifest(OUTPUT_DIR / 'manifest.json', manifest)
manifest


{'total_docs': 16950,
 'total_chars': 229433847,
 'after_dedup_docs': 16950,
 'after_filter_docs': 16799,
 'train_docs': 16463,
 'val_docs': 336,
 'params': {'train_ratio': 0.98,
  'min_chars': 50,
  'max_chars': 0,
  'unique_char_ratio_min': 0.01,
  'seed': 42}}

## プレビュー
前処理済みの先頭の文書を確認します。

In [19]:
print(train_docs[0][:1000] if train_docs else '(no docs)')


改造社の古木鉄太郎君の言ふには、「短歌は将来の文芸からとり残されるかどうか?」に就き、僕にも何か言へとのことである。僕は作歌上の素人たる故、再三古木君に断つたところ、素人なればこそ尋ねに来たと言ふ、即ちやむを得ずペンを執り、原稿用紙に向つて見るに、とり残されさうな気もして来れば、とり残されぬらしい気もして来る。
まづ明治大正の間のやうに偉い歌よみが沢山ゐれば、とり残したくともとり残されぬであらう。そこで将来も偉い詩人が生まれ、その詩人の感情を盛るのに短歌の形式を用ふるとすれば、やはりとり残されぬのに相違ない。するととり残されるかとり残されぬかを決するものは未だ生まれざる大詩人が短歌の形式を用ふるかどうかである。
偉い詩人が生まれるかどうかは誰も判然とは保証出来ぬ。しかしその又偉い詩人が短歌の形式を用ふるかどうかは幾分か見当のつかぬこともない。尤も僕等が何かの拍子に四つ這ひになつて見たいやうに、未だ生まれざる大詩人も何かの拍子に短歌の形式を用ふる気もちになるかも知れぬ。しかしそれは例外とし、まづ一般に短歌の形式が将来の詩人の感情を盛るに足るかどうかは考へられぬ筈である。
然るに元来短歌なるものは格別他の抒情詩と変りはない。変りのあるのは三十一文字に限られてゐる形式ばかりである。若し三十一文字と云ふ形式に限られてゐる為に、その又形式に纏綿した或短歌的情調の為に盛ることは出来ぬと云ふならば、それは明治大正の間の歌よみの仕事を無視したものであらう。たとへば斎藤氏や北原氏の歌は前人の少しも盛らなかつた感情を盛つてゐる筈である。しかし更に懐疑的になれば、明治大正の間の歌よみの短歌も或は猪口でシロツプを嘗めてゐると言はれるかも知れぬ。かう云ふ問題になつて来ると、素人の僕には見当がつかない。唯僕に言はせれば、たとへば斎藤氏や北原氏の短歌に或は猪口でシロツプを嘗めてゐるものがあるとしても、その又猪口の中のシロツプも愛するに足ると思ふだけである。
尤も物盛なれば必ず衰ふるは天命なれば、余り明治大正の間に偉い歌よみが出過ぎた為にそれ等の人人の耄碌したり死んでしまつたりした後の短歌は月並みになつてしまふかも知れぬ。それを将来の文芸からとり残されると云ふ意味に解釈すれば、或はとり残されると云ふ意味に解釈すれば、或はとり残されることもあるであらう。これは前にも書いたやうに作歌上の素人談義たるの